In [1]:
import torch
from iTransformer import iTransformer

import scipy.io
import numpy as np

file_path = '/home/rithvik/iitm2/csi-prediction/EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)
data = data['EV_re_im_split']
print(data.shape)

(50, 1000, 832)


In [2]:
model = iTransformer(
    num_variates = 832,
    lookback_len = 1000,                  # or the lookback length in the paper
    dim = 256,                          # model dimensions
    depth = 6,                          # depth
    heads = 8,                          # attention heads
    dim_head = 64,                      # head dimension
    pred_length = (5,),     # can be one prediction, or many
    num_tokens_per_variate = 1,         # experimental setting that projects each variate to more than one token. the idea is that the network can learn to divide up into time tokens for more granular attention across time. thanks to flash attention, you should be able to accommodate long sequence lengths just fine
    use_reversible_instance_norm = True # use reversible instance normalization, proposed here https://openreview.net/forum?id=cGDAkQo1C0p . may be redundant given the layernorms within iTransformer (and whatever else attention learns emergently on the first layer, prior to the first layernorm). if i come across some time, i'll gather up all the statistics across variates, project them, and condition the transformer a bit further. that makes more sense
)


/home/rithvik/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
data = torch.from_numpy(data).float()

In [4]:
preds = model(data)

In [13]:
preds[5]

tensor([[[-0.1187, -0.2174,  0.0034,  ...,  0.0242,  0.0633,  0.0953],
         [-0.0384,  0.0744, -0.0009,  ...,  0.1139,  0.0410,  0.1060],
         [ 0.0382, -0.0018,  0.0033,  ..., -0.0694,  0.0537,  0.1202],
         [ 0.0118, -0.0390, -0.0491,  ...,  0.1146, -0.0253, -0.0529],
         [ 0.0871,  0.0953,  0.0957,  ..., -0.0151, -0.0175, -0.0072]],

        [[-0.0682,  0.1956,  0.1479,  ...,  0.0208,  0.1254,  0.0691],
         [ 0.2183, -0.0943,  0.0959,  ...,  0.0056, -0.0369,  0.1261],
         [ 0.0362, -0.0638,  0.0322,  ...,  0.0769, -0.0202, -0.0428],
         [ 0.0874, -0.1144,  0.0632,  ...,  0.0152,  0.1154, -0.0619],
         [-0.0302,  0.0196, -0.0317,  ..., -0.0653,  0.0592, -0.0477]],

        [[-0.0775,  0.0430, -0.0264,  ...,  0.1098,  0.1095,  0.0565],
         [ 0.1970,  0.0269,  0.1882,  ...,  0.1035, -0.1654,  0.0505],
         [-0.1152,  0.0156, -0.1032,  ...,  0.0321,  0.0004,  0.0174],
         [-0.0007, -0.0139, -0.0999,  ...,  0.0723,  0.1736, -0.0240],
  

In [14]:
preds[5][0][0][2]

tensor(0.0034, grad_fn=<SelectBackward0>)

In [15]:
values=preds[5]
del preds

In [16]:
values.shape

torch.Size([50, 5, 832])